In [1]:
import xml.etree.cElementTree as et
from lxml import etree
import pandas as pd
import os
import re

In [2]:
# Import updated data
filePath='/home/ec2-user/SageMaker/data/corpus/regnews_jan2021/'
files=[]
for file in os.listdir(filePath):
    files.append(file)
print(len(files))
#print(files[0:5])

for file in files:
    if file.endswith('.xml'):
        pass
    else:
        print(file)

9278


In [3]:
# Function to print one XML example
def print_xml(file):
    tree = etree.parse(file)
    xml = etree.tostring(tree, encoding="unicode", pretty_print=True)
    print(xml)

In [4]:
print_xml(filePath+files[0])

<RECORD>
   <GOID>2467787339</GOID>
   
   <Obj>
      <SourceRollupType>Newspapers</SourceRollupType>
      <ObjectTypes>
         <other ObjectTypeOrigin="EXT">News</other>
         <mstar>News</mstar>
      </ObjectTypes>
      <ObjectRollupType>Articles - All Types</ObjectRollupType>
      <TitleAtt HTMLContent="true">
         <Title>JD Health Makes Strong Debut in Hong Kong; The city's stock market closes out its best year for listing activity in a decade, thanks in big part to Chinese technology companies</Title>
      </TitleAtt>
      <NumericDate>2020-12-08</NumericDate>
      <StartDate>2020-12-08</StartDate>
      <EndDate>2020-12-08</EndDate>
      <AlphaDate>Dec 8, 2020</AlphaDate>
      <Language>       
         <RawLang>English</RawLang>
     </Language>
      <Language IsPrimary="true">       
         <ISO>         
            <ISOCode>ENG</ISOCode>
            <ISOExpansion ISOCode="ENG">English</ISOExpansion>
         </ISO>
     </Language>
      <Copyright>
    

In [5]:
# Function to remove html tags from a string
def remove_html_tags(text):
    clean = re.compile('<.*?>')
    return re.sub(clean, '', text)

# Function to remove multiple spaces
def remove_spaces(text):
    text=re.sub(' +',' ',text).strip()
    return text

In [6]:
# Function to parse XML
def import_xml(ID, file):
    xmlp = et.XMLParser(encoding="UTF-8")
    parsed_xml = et.parse(file,parser=xmlp)
    root = parsed_xml.getroot()
    for child in root.findall('Obj'):
        lang=child.find('Language').find('RawLang').text
    if lang=='English':
        for child in root.findall('Obj'):
            type=child.find('ObjectTypes').find('mstar').text
            title=child.find('TitleAtt').find('Title').text
            try:
                startdate=child.find('StartDate').text
                enddate=child.find('EndDate').text
            except:
                startdate=child.find('NumericDate').text
                enddate=child.find('NumericDate').text

        if root.find('TextInfo')!=None:
            for node in root.iter('Text'):
                text=node.text
                text=remove_spaces(remove_html_tags(text))
                wordcount=node.get('WordCount')
        else:
            text=''
            wordcount=0

        for child in root.findall('DFS'):
            pubtitle=child.find('PubFrosting').find('Title').text
            sourcetype=child.find('PubFrosting').find('SourceType').text
    
        new_dic={'ID':ID,'Title':title,'Type':type,'StartDate':startdate,'EndDate':enddate,'Text':text,
            'TextWordCount':wordcount,'PubTitle':pubtitle, 'SourceType':sourcetype}
        df_xml=pd.DataFrame([new_dic])
    return df_xml

In [7]:
# Parse XML files
df=pd.DataFrame()
not_parsed=[]
for filename in files:
    if filename.endswith('.xml'):
        ID=filename.split('.xml')[0]
        file=filePath+filename
        try:
            new=import_xml(ID, file)
            df=df.append(new).reset_index(drop=True)
        except:
            not_parsed.append(filename)
            print('Could not parse:',filename)

In [8]:
print(df.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 9278 entries, 0 to 9277
Data columns (total 9 columns):
 #   Column         Non-Null Count  Dtype 
---  ------         --------------  ----- 
 0   ID             9278 non-null   object
 1   Title          9278 non-null   object
 2   Type           9278 non-null   object
 3   StartDate      9278 non-null   object
 4   EndDate        9278 non-null   object
 5   Text           9278 non-null   object
 6   TextWordCount  9278 non-null   object
 7   PubTitle       9278 non-null   object
 8   SourceType     9278 non-null   object
dtypes: object(9)
memory usage: 652.5+ KB
None


In [9]:
print(df.head())

           ID                                              Title  Type  \
0  2467787339  JD Health Makes Strong Debut in Hong Kong; The...  News   
1  2440267377  Banks' Retreat From Commodities Is Set to Dera...  News   
2  2473599411                    The Climate Desk Girds for 2021  News   
3  2465172996  New York Will Reopen Elementary Schools and Ph...  News   
4  2454308316  U.S. News: Justices Reject Pandemic-Tied Votin...  News   

    StartDate     EndDate                                               Text  \
0  2020-12-08  2020-12-08  Shares in one of China's top telemedicine prov...   
1  2020-09-05  2020-09-05  European banks have grown wary of financing co...   
2  2020-12-30  2020-12-30  After a year when climate-related disasters se...   
3  2020-11-29  2020-11-29  FILE -- Students outside of a Brooklyn public ...   
4  2020-10-27  2020-10-27  WASHINGTON -- The Supreme Court on Monday decl...   

  TextWordCount                      PubTitle  SourceType  
0           56

In [10]:
if len(not_parsed)>0:
    print(len(not_parsed))
    print(not_parsed)

In [11]:
df.to_pickle('/home/ec2-user/SageMaker/New Uncertainty/Update Data/parsed_xml_update.pkl')

In [12]:
df=pd.read_pickle('/home/ec2-user/SageMaker/New Uncertainty/Update Data/parsed_xml_update.pkl')
print(df.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 9278 entries, 0 to 9277
Data columns (total 9 columns):
 #   Column         Non-Null Count  Dtype 
---  ------         --------------  ----- 
 0   ID             9278 non-null   object
 1   Title          9278 non-null   object
 2   Type           9278 non-null   object
 3   StartDate      9278 non-null   object
 4   EndDate        9278 non-null   object
 5   Text           9278 non-null   object
 6   TextWordCount  9278 non-null   object
 7   PubTitle       9278 non-null   object
 8   SourceType     9278 non-null   object
dtypes: object(9)
memory usage: 652.5+ KB
None


In [13]:
print(df['SourceType'].value_counts())
print(df['Type'].value_counts())

Newspapers    9278
Name: SourceType, dtype: int64
News    9278
Name: Type, dtype: int64


In [14]:
# Convert dates
df['StartDate']=df['StartDate'].astype('datetime64[ns]')
df['Year']=df['StartDate'].astype('datetime64[ns]').dt.year
df['Month']=df['StartDate'].astype('datetime64[ns]').dt.month

In [15]:
# Consolidate newspaper names
df.loc[(df['PubTitle']=='Boston Globe (pre-1997 Fulltext)') | (df['PubTitle']=='Boston Globe') | 
       (df['PubTitle']=='Boston Globe (Online)'),'Newspaper']='Boston Globe'
df.loc[(df['PubTitle']=='Wall Street Journal') | (df['PubTitle']=='Wall Street Journal (Online)'),
    'Newspaper']='Wall Street Journal'
df.loc[(df['PubTitle']=='USA TODAY (pre-1997 Fulltext)') | (df['PubTitle']=='USA TODAY') | 
       (df['PubTitle']=='USA Today (Online)'),'Newspaper']='USA Today'
df.loc[(df['PubTitle']=='Chicago Tribune (pre-1997 Fulltext)') | (df['PubTitle']=='Chicago Tribune') | 
       (df['PubTitle']=='Chicago Tribune (Online)'),'Newspaper']='Chicago Tribune'
df.loc[(df['PubTitle']=='Los Angeles Times') | (df['PubTitle']=='Los Angeles Times (pre-1997 Fulltext)') | 
        (df['PubTitle']=='Los Angeles Times (Online)'),'Newspaper']='Los Angeles Times'
df.loc[(df['PubTitle']=='New York Times') | (df['PubTitle']=='New York Times (Online)'),'Newspaper']='New York Times'
df.loc[(df['PubTitle']=='The Washington Post') | (df['PubTitle']=='The Washington Post (pre-1997 Fulltext)') | 
       (df['PubTitle']=='The Washington Post (Online)'),'Newspaper']='The Washington Post'

In [16]:
print(df['Newspaper'].value_counts())

Wall Street Journal    3394
New York Times         1675
The Washington Post    1220
USA Today              1131
Boston Globe            984
Chicago Tribune         459
Los Angeles Times       415
Name: Newspaper, dtype: int64


In [17]:
df=df.sort_values(['Newspaper','StartDate','Title']).reset_index(drop=True)

In [18]:
df.to_pickle('/home/ec2-user/SageMaker/New Uncertainty/Update Data/parsed_xml_update.pkl')